In [1]:
import pandas as pd
import nltk
import string

import sqlite3
import warnings
warnings.filterwarnings("ignore")
#Membaut connection terhadap database

from sqlite3 import Error
db_file = 'DBnews.db'
conn = None
try:
    conn = sqlite3.connect(db_file)
except Error as e:
    print(e)

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
dataset = pd.read_sql_query("select comment from t_comments", conn)
dataset

,comment
0,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA"
1,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�
2,Salut buat Gus Miftah
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat
4,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..
...,...
598092,212 nggak pernah ngadain acara beginian ya....terus program mereka dalam keagamaan apa ya....kalo mereka ngadain nya abis saya nulis ini...berarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong
598093,TBG... Sayank.. salah pilih kendraaan..�
598094,Saya bersama TGB pilih Jokowi�
598095,"Hijrah dari hoax ke non hoax, dari ngawur ke bener, dari saling menjelekkan ke saling puji dan hormat!"


In [4]:
#memperkecil data yang akan diproses
#dataset diperkecil menjadi 1000 untuk mempercepat proses yang akan dilakukan
#atau memperkecil waktu yang digunakan untuk melakukan text processing
dataset = dataset[:1000]

len(dataset)

1000

In [5]:
# Case Folding: Lowercase
dataset['comment'] = dataset['comment'].str.lower()

# Remove Punctuation: Menghapus tanda baca
punctuation = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

dataset['comment'] = dataset['comment'].apply(lambda text: remove_punctuation(text))

# Non Alphabet Removal: Remove number and <?>
dataset['comment'] = dataset['comment'].str.replace(r'[^a-zA-Z ]\s?',r'', regex=True)

In [6]:
dataset['tokenized_comment'] = dataset.apply(lambda row: nltk.word_tokenize(row['comment']), axis=1)

In [7]:
dataset

,comment,tokenized_comment
0,wkwkwk gak sekalian pak selawat ke dpr kan banyak koruptor disana,"[wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]"
1,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa,"[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]"
2,salut buat gus miftah,"[salut, buat, gus, miftah]"
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo sunan kali jagasmua ada cara demi kebaikan asal jng ajk dg cara berjinah lakukan dg porsi dan sikon yg tepat,"[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]"
4,ada adabnya sholawat lebih baik ajak ke majelis ilmu,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...,...
995,bagus jangan ikut orang jahat saudarapapua merasakan pembangunan,"[bagus, jangan, ikut, orang, jahat, saudarapapua, merasakan, pembangunan]"
996,apa kata gerinda cs ya hayo apa,"[apa, kata, gerinda, cs, ya, hayo, apa]"
997,karena papu maju,"[karena, papu, maju]"
998,mbah mien langsung kejangkejang,"[mbah, mien, langsung, kejangkejang]"


In [8]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

dataset['stem_comment'] = dataset['tokenized_comment'].apply(lambda x: [stemmer.stem(y) for y in x])

In [9]:
dataset

,comment,tokenized_comment,stem_comment
0,wkwkwk gak sekalian pak selawat ke dpr kan banyak koruptor disana,"[wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]","[wkwkwk, gak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]"
1,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa,"[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]","[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]"
2,salut buat gus miftah,"[salut, buat, gus, miftah]","[salut, buat, gus, miftah]"
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo sunan kali jagasmua ada cara demi kebaikan asal jng ajk dg cara berjinah lakukan dg porsi dan sikon yg tepat,"[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]","[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jng, ajk, dg, cara, jah, laku, dg, porsi, dan, sikon, yg, tepat]"
4,ada adabnya sholawat lebih baik ajak ke majelis ilmu,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]","[ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...,...,...
995,bagus jangan ikut orang jahat saudarapapua merasakan pembangunan,"[bagus, jangan, ikut, orang, jahat, saudarapapua, merasakan, pembangunan]","[bagus, jangan, ikut, orang, jahat, saudarapapua, rasa, bangun]"
996,apa kata gerinda cs ya hayo apa,"[apa, kata, gerinda, cs, ya, hayo, apa]","[apa, kata, gerinda, cs, ya, hayo, apa]"
997,karena papu maju,"[karena, papu, maju]","[karena, papu, maju]"
998,mbah mien langsung kejangkejang,"[mbah, mien, langsung, kejangkejang]","[mbah, mien, langsung, kejangkejang]"


In [10]:
print(dataset['stem_comment'])

0      [wkwkwk, gak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                               
1      [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                            
2      [salut, buat, gus, miftah]                                                                                                                                                                              
3      [yg, nyinyir, gak, pernah, lihat, dan, baca, atw, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jng, ajk, dg, cara, jah, laku, dg, porsi, dan, sikon, yg, tepat]
4      [ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]                                                                                                      

In [11]:
#test output matrix

print(dataset['stem_comment'][5][3])

jalan


In [12]:
for i in range(len(dataset['stem_comment'])):
    dataset['stem_comment'][i] = ' '.join(dataset['stem_comment'][i])
    
    print(dataset['stem_comment'])

0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                                                           
1      [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                            
2      [salut, buat, gus, miftah]                                                                                                                                                                              
3      [yg, nyinyir, gak, pernah, lihat, dan, baca, atw, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jng, ajk, dg, cara, jah, laku, dg, porsi, dan, sikon, yg, tepat]
4      [ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]                                                                                                      

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                                                                             

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

...

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

Name: stem_comment, Length: 1000, dtype: object
0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                             

0      wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                          
1      mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                            
2      salut buat gus miftah                                                                                                                                                  
3      yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4      ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                                      
                             ...                                                                                             

In [13]:
dtset_stem = pd.read_csv('sample_data_stem.csv')
dtset_stem

,stem_comment
0,wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana
1,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa
2,salut buat gus miftah
3,yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4,ada adab sholawat lebih baik ajak ke majelis ilmu
5,tiap ulama punya jalan dan cara dakwah masingjika kamu lihat kemunkaran lawan dengan tanganmukekuatanmu jika tidak mampu lawan dengan mulut jika tidak mampu juga dengan hati itu lemah lemah iman itu sudah jadi wajib para ulama kalo yg komen masih ada yg nyinyirin ulama arti otak sudah disisipin politik
6,umpanin nocannya yak
7,itu ceramah yg benar
8,besok bugil aja selawat
9,beda gus miftah dengan fpi hti bagai surga dan neraka di surga penuh kasih di neraka penuh hancur


In [14]:
df_colloquial_indonesian_lexicon = pd.read_csv('colloquial-indonesian-lexicon.csv')
df_colloquial_indonesian_lexicon = df_colloquial_indonesian_lexicon.drop(['In-dictionary', 'context', 'category1', 'category2', 'category3'], axis = 1)
df_colloquial_indonesian_lexicon.head(1000)

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa
...,...,...
995,fto,foto
996,gtu,begitu
997,jg,juga
998,bgt,banget


In [15]:
df_kamuskateglo = pd.read_csv('kamus_alay.csv', delimiter = "\t", header=None, names=["slang", "normal"])
df_kamuskateglo.head(1000)

,slang,normal
0,"slang,formal",NaN
1,"woww,wow",NaN
2,"aminn,amin",NaN
3,"met,selamat",NaN
4,"netaas,menetas",NaN
...,...,...
995,"baju'a,bajunya",NaN
996,"fto,foto",NaN
997,"gtu,begitu",NaN
998,"jg,juga",NaN


https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe

In [16]:
import json

In [17]:
kateglo_dictionary = json.load(open('data_kamus_kateglo.json'))
df_kateglo_dictionary = pd.DataFrame(kateglo_dictionary)
df_kateglo_dictionary = df_kateglo_dictionary.drop(['kelas', 'type'], axis = 1)
df_kateglo_dictionary.head(1000)

,lema,nilai
0,-an,bt 1 hasil tindakan; 2 yang biasa dikenai tindakan; yang di-; 3 tempat; lokasi; 4 alat untuk mengukur atau menghitung; 5 hal atau cara; 6 kumpulan; gugusan; 7 yang mempunyai atau mengandung; 8 (cak) yang menyerupai; 9 sesuatu yang mempunyai sifat; 10 (cak) intensitas; 11 (cak) melakukan sesuatu yang pelaku atau tindakannya banyak (saling); 12 kegiatan yang berkenaan dengan pelaku atau tindakan yang banyak;
1,-anda,bt menyatakan hormat atau kekeluargaan;
2,-asi,bt → -isasi;
3,-da,bt → -anda;
4,-el-,bt 1 yang melakukan; 2 alat; 3 benda yang kelopak; 4 kumpulan; 5 terjadi berlaku atau melakukan;
...,...,...
995,akuifer,n (Geo) lapisan kulit bumi berpori yang dapat menahan air dan terletak di antara dua lapisan yang kedap air;
996,akuifer artesis,n (Tan) akuifer yang airnya dibatasi oleh dasar yang relatif kedap air sehingga mendapat tekanan yang cukup untuk menaikkan air sumur ke atas puncak akuifer;
997,akuisisi,n 1 perolehan; pemerolehan; 2 masukan (tentang data komputer); 3 pemindahan kepemilikan perusahaan atau aset (dalam industri perbankan terjadi apabila pembelian saham di atas 50%); pengambilalihan kepemilikan perusahaan atau aset; 4 (Ek) cara memperbesar perusahaan dengan cara memiliki perusahaan lain;
998,akuk,n penyakit ayam (yang menyebabkan ayam tertunduk atau meromok);


In [18]:
df_kateglo_dictionary.shape

(72253, 2)

In [19]:
kateglo_abbreviation_acronym = json.load(open('data_singkatan_dan_akronim_kateglo.json'))
df_kateglo_abbreviation_acronym = pd.DataFrame.from_dict(kateglo_abbreviation_acronym, orient="index").reset_index()
df_kateglo_abbreviation_acronym.columns = ['singkatan', 'bahasa_indonesia']
df_kateglo_abbreviation_acronym.head(10)

,singkatan,bahasa_indonesia
0,a.d.,atas dasar
1,a.h.,None
2,a.i.,ad interim
3,a.l.,antara lain
4,a.m.,sebelum tengah hari
5,a.n.,atas nama
6,a.n.b.,atas nama beliau
7,a.p.,atas perintah
8,a.s.,alaihi salam
9,a/o,None


In [20]:
df_kateglo_abbreviation_acronym.shape

(3389, 2)

## Mapping word to dictionary

In [21]:
contoh_dataset = dtset_stem['stem_comment']
contoh_dataset

0     wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                                                                                                                                                                  
1     mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                                                                                                                                                                    
2     salut buat gus miftah                                                                                                                                                                                                                                                                                          
3     yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah wal

https://stackoverflow.com/questions/54549482/replacing-values-in-one-pandas-dataframe-with-values-from-another-dataframe

In [22]:
contoh_dataset = dtset_stem

In [23]:
contoh_dataset

,stem_comment
0,wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana
1,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa
2,salut buat gus miftah
3,yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat
4,ada adab sholawat lebih baik ajak ke majelis ilmu
5,tiap ulama punya jalan dan cara dakwah masingjika kamu lihat kemunkaran lawan dengan tanganmukekuatanmu jika tidak mampu lawan dengan mulut jika tidak mampu juga dengan hati itu lemah lemah iman itu sudah jadi wajib para ulama kalo yg komen masih ada yg nyinyirin ulama arti otak sudah disisipin politik
6,umpanin nocannya yak
7,itu ceramah yg benar
8,besok bugil aja selawat
9,beda gus miftah dengan fpi hti bagai surga dan neraka di surga penuh kasih di neraka penuh hancur


In [24]:
# membandingkan kamus_alay dengan colloquial_indonesian_lexicon
dtset_kamus_alay = pd.read_csv('colloquial-indonesian-lexicon.csv')
dtset_kamus_alay

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,"Selamat ulang tahun kakak tulus semoga panjang umur kakak,sehat selalu juga,murah rezeki ya kakak dan jangan sombong kalo sukses di karirnya aminn @tulusm",elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @tulusm,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @tulusm,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
...,...,...,...,...,...,...,...
15001,gataunya,enggak taunya,0,Ini kaya nenek2 ya beb gataunya agnezz @yugimellasih,akronim,0,0
15002,gtau,enggak tau,0,"Stidaknya mrka may berkarya Dan berusaha yg trbaikk bkan kmauan mrka tpi mrka punn mengikutii atassan mrka,so klauu gtau jngnn so ngjudge orng kya gtu,kya sendirinyya punya frestasiii ajjh @meilisa_kyungsoo_",akronim,abreviasi,0
15003,gatau,enggak tau,0,Ih gatau malu,akronim,0,0
15004,fans2,fan-fan,0,Jkt48 adalah tempat di mana sesama fans saling menghujat..men judge sesama fans.. dimana petinggi2 fans club slalu memandang sebelah mata para wota2 lain..yg hanya ingin membuktikan bahwa diri nya adalah yg terbaik di antara fans2 yg lain.. itu alasan knpa aku lebih memilih jalur indefendet ketimbang mengikuti fans club jkt48. Karena mencintai dan mendukung jkt48 gak harus melalui fans club kan ??? Karena fansclub hanya tempat2 orang yg kyk taik.. yg slalu menghujat sesama fans jkt48.. fuck to wota .!!! I love jkt48 .!!!!! wota anak2 sampah..!!!,reduplikasi,naturalisasi,0


In [25]:
print(list(dtset_kamus_alay['slang']))

['woww', 'aminn', 'met', 'netaas', 'keberpa', 'eeeehhhh', 'kata2nyaaa', 'hallo', 'kaka', 'ka', 'daah', 'aaaaahhhh', 'yaa', 'smga', 'slalu', 'amiin', 'kk', 'trus', 'kk', 'sii', 'nyenengin', 'bgt', 'gemess', 'akuuu', 'jgn', 'yaa', 'udah', 'gitu', 'aja', 'gemesiin', 'menyenangkn', 'rb', 'akau', 'saranin', 'nemuin', 'yah', 'met', 'mnis2nya', 'gtu', 'tu', 'idolaa', 'bangett', 'kaakk', 'baaiikkk', 'telaatt', 'kaak', 'yg', 'lg', 'skarang', 'yg', 'sempet', 'karna', 'ngga', 'karna', 'mantep', 'loohhh', 'yeee', 'baguuuuus', 'sampe', 'ngikutin', 'gayany', 'ngikutin', 'lg', 'bnk', 'awaas', 'siip', 'abizzzz', 'kerrrrennnn', 'sayaa', 'pengen', 'kpan', 'bgt', 'sma', 'sdkit', 'bngt', 'lgu', 'bener2', 'kharismanya', 'aq', 'yg', 'jdi', 'lgi', 'gk', 'kek', 'lgunya', 'kpan', 'kereeeeennnn', 'nyamain', 'slalu', 'lakik', 'siippp', 'bkn', 'nyangka', 'bangettt', 'kereeen', 'wihhhhhh', 'tp', 'inget', 'bingitsss', 'dmn', 'bird', 'kaloo', 'gimna', 'tuuh', 'nyebur', 'yng', 'dbuat', 'kn', 'org', 'pingin', 'bgt', '

In [26]:
for i in range(len(contoh_dataset['stem_comment'])):
    contoh_dataset['stem_comment'][i] = contoh_dataset['stem_comment'][i].split(' ')

In [27]:
dtset_stem['stem_comment']

0     [wkwkwk, gak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                                                                                                                                                                        
1     [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                                                                                                                                                     
2     [salut, buat, gus, miftah]                                                                                                                                                                                                                                                        

In [28]:
for i in range(len(dtset_stem['stem_comment'])):
    for j in range(len(dtset_stem['stem_comment'][i])):
        try:
            k_awal = dtset_stem['stem_comment'][i][j]
            dtset_stem['stem_comment'][i][j] = list(dtset_kamus_alay['formal'])[(list(dtset_kamus_alay['slang']).index(dtset_stem['stem_comment'][i][j]))]
            k_akhir = dtset_stem['stem_comment'][i][j] # kata yg diubah
            print(dtset_stem['stem_comment'][i][j])
        except:
            pass
#             print(dtset_stem['stem_comment'][i][j])

enggak
baru
yang
kalau
ada
deh
yang
enggak
atau
jangan
dengan
dengan
yang
yang
yang
nyinyir
ya
yang
saja
kenapa
yang
keren
dapat
dari
yang
saja


In [29]:
dtset_stem['stem_comment']

0     [wkwkwk, enggak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                                                                                                                                                                       
1     [mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]                                                                                                                                                                                                                                                             
2     [salut, buat, gus, miftah]                                                                                                                                                                                                                                                    